# Guider mode image reduction
Craig Lage - 02-Aug-24

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath

In [ ]:
dayObs = 20240807
seqNum = 9
raft = 'R22'
ccd = 'S00'
filename = f"s3://rubin-summit/LSSTComCam/{dayObs}/CC_O_{dayObs}_{seqNum:06d}/CC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)

In [ ]:
hdu_list[0].header

In [ ]:
hdu_list[4].header

In [ ]:
data = hdu_list[4].data

In [ ]:
plt.title(f"ComCam guider mode", fontsize=12)
plt.imshow(data, interpolation='nearest')
plt.colorbar()

In [ ]:
expId = int(f"{dayObs}{seqNum:05d}")
dirName = f"/home/c/cslage/u/ComCam/guider_mode/movie_{expId}"
%mkdir -p {dirName}
movieName = f"Guider_{expId}.mp4"
print(movieName)

# Build the individual frames
fig = plt.figure(figsize=(8, 8))
nStamps = int(hdu_list[0].header['N_STAMPS'])
for n in range(nStamps):
    data = hdu_list[2 * n + 2].data
    plt.imshow(data, interpolation='nearest')
    plt.colorbar()
    plt.title(f"ComCam guider mode {expId}, Frame {n+1}", fontsize=12)
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()
    print(f"Finished {2 * n + 2}")


In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()

In [ ]:
data = (hdu_list[25].data['rawStamp'][0]).astype('>u4')
data.byteswap(inplace=True)
xor = 0x1ffff
size = 50 * 53
out = np.zeros([16, size], dtype=int)
image_out = np.zeros([16, 50,50], dtype=int)
for n in range(size):
    res = ''
    for i in range(9):
        d = data[(size-n)*9-i-1]
        d = format(d, '#034b')
        d = d.split('b')[1]
        res += d
    for i in range(16):
        bin_value = res[i*18:(i+1)*18]
        int_value = int(bin_value, 2)
        final_value = int_value ^ xor
        out[i,n] = final_value  
for i in range(16):
    reshaped = out[i,:].reshape(50,53)
    image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:50]))

In [ ]:
fig, axs = plt.subplots(4,4,figsize=(10,10))
plt.subplots_adjust(wspace=.7, hspace=-0.2)
for i in range(4):
    for j in range(4):
        n = i + 4 * j
        axs[i][j].set_title(f"segment {n}")
        im = axs[i][j].imshow(image_out[n], interpolation='nearest', origin='lower')
        divider = make_axes_locatable(axs[i][j])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax)
